In [1]:
# load agent from json
# create agent and write to disk

In [ ]:
from __future__ import annotations
import os
from dotenv import load_dotenv

import asyncio, pathlib, json, uuid
from typing import Dict, List
from pydantic import BaseModel, Field
from agents import Agent, Runner

In [ ]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI API KEY IS NOT FOUND IN THE ENVIORMENT VARIABLES")

In [ ]:
class AgentSpec(BaseModel):
    """元生成器产出的结构化描述"""
    name: str
    instructions: str
    label: str | None = None          # 可选：给 Registry 用


META_AGENT = Agent(
    name="MetaGenerator",
    instructions=(
        "根据用户提供的任务描述，输出完整的 Agent 设计。"
        "只要保证字段齐全即可。"
    ),
    output_type=AgentSpec,
)


async def generate_agent_spec(task_desc: str) -> AgentSpec:
    res = await Runner.run(META_AGENT, task_desc)
    return res.final_output 


class DynamicAgent:
    def __init__(self):
        self._by_label: Dict[str, Agent] = {}
        self.orchestrator = Agent(
            name="Orchestrator",
            instructions=(
                "你是一名通用助手。根据用户需求，调用最匹配的工具或 hand-off。"
                "若需要多个步骤，可以依次调用多个工具。"
                "你自己不解决问题，永远通过工具/hand-off 完成。"
            ),
            tools=[],      # 动态追加
            handoffs=[],   # 如需 hand-off 也可动态追加
        )
        self._load_existing()

    async def ensure_tool(self, label: str, task_desc: str):
        if label in self._by_label:
            return

        spec = await generate_agent_spec(task_desc)
        agent = Agent(name=spec.name, instructions=spec.instructions)
        self._register(agent, label or spec.label or spec.name)

    async def solve(self, query: str) -> str:
        # 1) 标签化
        label = await classify(query)
        # 2) 确保有对应工具
        await self.ensure_tool(label, f"请为任务“{label}”设计 agent：{query}")
        # 3) Orchestrator 调度
        res = await Runner.run(self.orchestrator, query)
        return res.final_output

    def _register(self, agent: Agent, label_hint: str | None, persist: bool = True):
        label = label_hint or agent.name.lower().replace(" ", "_")
        self._by_label[label] = agent
        # 本示例全部挂到 tools；如想“完全交接”可用 hand-off
        self.orchestrator.tools.append(
            agent.as_tool(
                tool_name=f"use_{label}",
                tool_description=f"Handle task: {label}",
            )
        )
        if persist:
            save_cfg(AgentCfg(name=agent.name, instructions=agent.instructions))  

    
    async def _create_and_register(self):
        pass

    

    

IndentationError: expected an indented block (1258766434.py, line 12)